In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from unicodedata import normalize
%matplotlib inline

In [2]:
df = pd.read_excel('all.xls')

In [3]:
df.head()

,Mephi_Authors,Title,Source,Year,Volume,Issue,Pages,Collab,DOI,Scopus_Cited,...,Exists_Scopus,Exists_Wos,Date_Scopus,Date_Wos,URL,Center,Kaf,SAE,ASJC,Quartile
0,"Azarkin,M, Dremin,I, Leonidov,A, Antonov,A., B...",Combined Measurement of the Higgs Boson Mass i...,PHYSICAL REVIEW LETTERS,2015,114,19,NaN,ATLAS Collaboration,10.1103/PhysRevLett.114.191803,278.0,...,1,1,20160806,20150611,https://scopus.com/alert/results/record.uri?AI...,"Фундаментальные исследования и физика частиц,...","040 (физики элементарных частиц), 040 (физик...","ИЯФИТ, ИЯФИТ, ИЯФИТ, , ИЯФИТ, ИЯФИТ, ИЯФИТ, ИЯ...","Medicine (all), Physics and Astronomy (all),",Q1
1,"Mizuk,R,",QCD and strongly coupled gauge theories: chall...,EUROPEAN PHYSICAL JOURNAL C,2014,74,10,NaN,NaN,10.1140/epjc/s10052-014-2981-5,171.0,...,1,1,20150112,20150108,https://scopus.com/alert/results/record.url?AI...,"Наноструктурная электроника,",007 (экспериментальной ядерной физики и космо...,"ИЯФИТ, , ,","Engineering (miscellaneous), Physics and Astro...",Q1
2,"Shelyakov,A.,",Violation of the sequence of martensite crysta...,Phase Transitions,2017,90,3,289-298,0,10.1080/01411594.2016.1180517,0.0,...,1,0,20160513,0,https://scopus.com/alert/results/record.uri?AI...,"Плазменные, лазерные исследования и технологии,","070 (физики твердого тела и наносистем), , ,","ЛАПЛАЗ, , ,","Instrumentation, Materials Science (all),",Q2
3,"Kudryashov,N.A., Volkov,A.K.,",The fifth-order partial differential equation ...,Communications in Nonlinear Science and Numeri...,2017,42,NaN,491-501,NaN,10.1016/j.cnsns.2016.06.003,2.0,...,1,1,20160702,20160923,https://scopus.com/alert/results/record.uri?AI...,"Прикладная математика и теоретическая физика,...","031 (прикладной математики), Факультет: (Дру...","ИИКС, , , , , ,","Numerical Analysis, Modeling and Simulation, A...",Q1
4,"Minin,P.,",Sharpness estimation in facial images by spect...,"Signal, Image and Video Processing",2017,11,1,163-170,NaN,10.1007/s11760-016-0915-4,0.0,...,1,1,20160715,20170216,https://scopus.com/alert/results/record.uri?AI...,",","Нет данных об авторе, , ,",", , ,","Signal Processing, Electrical and Electronic E...",Q2


## Count publications for ech kaf in each category

In [4]:
def get_elements(regex, string, default='N/A'):
    #Returns a list of substrings that match the regex attribute
    try: 
        result = re.findall(regex, string)
    except TypeError:
        result = [default,]
    return result

In [5]:
kafs_regex = '[\D][0-9][0-9][0-9][\D]|[0-9][0-9][0-9][0-9]|Нет данных об авторе' # ADD "not numbers"

def make_list(regex, data_frame, label):
    #Returns a list of unique elements matching the regex and 
    #pesented in all rows of specified coloumn (label) of the data_frame
    seq = []
    for field in data_frame[label]:
        elements = get_elements(regex, field)
        for el in elements:
            try:
                seq.index(el)
            except ValueError:
                seq.append(el)            
    return seq

def make_dict(regex, data_frame, label, init=0):
    kaf_seq = make_list(regex, data_frame, label)
    unique_kaf_dict = dict.fromkeys(kaf_seq, init)
    return unique_kaf_dict

In [6]:
def is_inside(list_to_check, val_to_find):
    try:
        list_to_check.index(val_to_find)
    except (ValueError,AttributeError) as e:
        return False
    else:
        return True

In [10]:
def count_by_category(data_frame, category, cat_label, label='Kaf'):
    #returns a dict for Q with publications amount for each kaf
    unique_kaf_dict = make_dict(kafs_regex, df, label='Kaf')
    for idx,row in enumerate(data_frame[label]):
        #if data_frame[cat_label][idx]==category: # NOT SUFFICIENT as there can be more than one category in a row
        if is_inside(data_frame[cat_label][idx], category):
            kafs_found = get_elements(kafs_regex, row)
            for kaf in unique_kaf_dict.keys():
                try:
                    kafs_found.index(kaf)
                except ValueError:
                    {}
                else:
                    unique_kaf_dict[kaf] += 1
    return unique_kaf_dict

In [13]:
quartile_regex = 'Q[1-4]|Нет квартиля'
field_of_science_regex = "[\S|\s]+?[,]|[\S|\s]+"

label = 'ASJC'

def make_dict_by_category(regex, data_frame, cat_label='Quartile'):
    #Returns a dictionary where keys are the category values 
    #found in data_frame.
    #Values of the returned dict are dicts of counted publications 
    #for corresponding category
    result = {}
    seq = make_list(regex, data_frame, cat_label)
    for category in seq:
        result[category] = count_by_category(data_frame, category, cat_label)
    return result

counted_by_field = make_dict_by_category(field_of_science_regex, df, label)

### Writing results itno xls file

In [14]:
result = pd.DataFrame(counted_by_field)
result.to_csv('counted_by_ASJC.csv')

### Plot

In [ ]:
fig = plt.figure()

category = 'Q4'
plot = plt.pie(list(publications[category].values()), labels=publications[category].keys(), shadow=True, labeldistance=1.3)

## Count number of interactions between kafs


In [ ]:
kafs_regex = '[0-9][0-9][0-9]|[0-9][0-9][0-9][0-9]'
def is_presented(list_to_inspect, kaf_to_find):
    try:
        list_to_inspect.index(kaf_to_find)
    except ValueError:
        return False
    else:
        return True

def count_unique_elements_for(kaf):
    result_dict = make_dict(kafs_regex, df, 'Kaf')
    for row in df['Kaf']:
        kafs_found = get_elements(kafs_regex, row)
        if is_presented(kafs_found, kaf):
            for category in result_dict.keys():
                result_dict[category] += 1*is_presented(kafs_found, category)
    return result_dict

def count_interactions(data_frame, label):
    # Returns a dict of dicts with number of interactions with each kaf
    kaf_dict = make_dict(kafs_regex, df, label='Kaf')
    for kaf in kaf_dict.keys():
        kaf_dict[kaf] = count_unique_elements_for(kaf)
        delete_NA = kaf_dict[kaf].pop('N/A')
    return kaf_dict

result = count_interactions(df, 'Kaf')

## Writing results itno xls file

In [ ]:
result = pd.DataFrame(result)
result.to_csv('result.csv')